In [1]:
# initial setup
%run "../../../common/0_notebooks_base_setup.py"


/Users/csuarezgurruchaga/Desktop/Digital-House/CLASE_41/dsad_2021/common
default checking
Running command `conda list`... ok
jupyterlab=2.2.6 already installed
pandas=1.1.5 already installed
bokeh=2.2.3 already installed
seaborn=0.11.0 already installed
matplotlib=3.3.2 already installed
ipywidgets=7.5.1 already installed
pytest=6.2.1 already installed
chardet=4.0.0 already installed
psutil=5.7.2 already installed
scipy=1.5.2 already installed
statsmodels=0.12.1 already installed
scikit-learn=0.23.2 already installed
xlrd=2.0.1 already installed
Running command `conda install --yes nltk=3.5.0`... ok
Solving environment: ...working... done

# All requested packages already installed.


unidecode=1.1.1 already installed
pydotplus=2.0.2 already installed
pandas-datareader=0.9.0 already installed
flask=1.1.2 already installed


---

<img src='../../../common/logo_DH.png' align='left' width=35%/>


# <h1><center><ins>PIPELINE</ins></center></h1>
<h1><center>Checkpoint —solución— :</center></h1>
<img src="img/00_pipelines.jpg" alt="Drawing" style="width: 400px;"/>

<a id="tabla_contenidos"></a> 
## Tabla de Contenidos

### <a href='#section_objetivos'>0. Objetivos de la Notebook</a>


### <a href='#section_repaso'>1. Lo que tenés que hacer es...</a>
- #### <a href='#section_reg_vs_clas'>1.1 ...cargar el dataset y preparar los datos:</a>
- #### <a href='#1.2'>1.2 ...aplicar el Pipeline+GridSearchCV:</a>

<a id="section_objetivos"></a> 
## 0. Objetivos de la Notebook

<div id="caja11" style="float:left;width: 100%;">
  <div style="float:left;width: 9%;"><img src="../../../common/icons/haciendo_foco.png" style="align:left"/> </div>
  <br>
  <div style="float:left;width: 85%;">
      <label>Poner en práctica lo aprendido en la notebook de práctica guiada.</label>
  <div style="float:left;width: 85%;">
      <label>Es importante que antes de la clase resuelvan esta notebook ya que es fundamental que sepan utilizar las herramientas que vimos en la práctica guiada para después trabajar el caso práctico en la clase.</label>        
</div>    
</div>

<a href='#tabla_contenidos'>Volver a TOC</a>

<a id="section_repaso"></a> 
## 1. Lo que tenés que hacer es...

<a id="section_reg_vs_clas"></a>
### 1.1 ...cargar el dataset y preparar los datos:

Vamos a probar cómo armar `pipeline` combinado con `GridSearchCV`. Para ello vamos a volver a utilizar el dataset sobre la calidad de vinos que puede encontrarse en la página de Kaggle: https://www.kaggle.com/uciml/red-wine-quality-cortez-et-al-2009

El dataset está asociado a variantes rojas de un vino Portuguez. En la página de kaggle pueden encontrar la descripción de qué significan todas las variables. 

Importemos y exploremos un poco el dataset: 

In [2]:
import pandas as pd

df = pd.read_csv('../Data/winequality-red.csv',sep=';')
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


Binarizamos la calidad del vino (columna quality) tomando como criterio arbitrario, todos los vinos que tengan una calidad igual o mayor a 6 los vamos a considerar de buena calidad (les pondemos un 1 como etiqueta), mientras que los que sean inferiores a 6 los consideraremos de baja calidad (y los etiquetaremos con un 0):

In [4]:
df['quality'].value_counts()

5    681
6    638
7    199
4     53
8     18
3     10
Name: quality, dtype: int64

In [5]:
df['quality_bin']=df['quality'].apply(lambda x: 1 if x>=6 else 0)
df['quality_bin'].value_counts(normalize=True)

1    0.534709
0    0.465291
Name: quality_bin, dtype: float64

Separar la `X` (todas las features menos **quality** ni **quality_bin**) y la `y` (**quality_bin**), y separar los datos en aquellos que se van a usar para el entrenamiento y los que servirán para testear el modelo.

In [6]:
from sklearn.model_selection import train_test_split

# Separamos las variables independientes de la target
X=df.drop(columns=['quality','quality_bin'])
y=df['quality_bin']

# Dividimos los datos en el set de train y el de test: 
X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.3, random_state=30, stratify=y)
display(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(1119, 11)

(480, 11)

(1119,)

(480,)

<a href='#tabla_contenidos'>Volver a TOC</a>

<a id="1.2"></a>
### 1.2 ...aplicar el Pipeline+GridSearchCV:

Ahora que los datos están preparados, utilizando `Pipeline`+`GridSearchCV`, armar un proceso general en el que:

**a-** se seleccionen todas las features menos **residual sugar**

**b-** se normalicen los datos

**c-** se aplique una `LogistiRegression`


Con el `GridSearchCV`, el objetivo es explorar si el: en el primer paso **(a)** es mejor seleccionar o no hacer la selección; en el segundo paso **(b)** cuál es el mejor escalador si `StandardScaler` o `MinMaxScaler`, y por último en el paso **(c)** cuál es el valor óptimo del hiperparámetro C de la `LogisticRegression`. 

**(i)** Crear la clase para seleccionar las features:

In [7]:
from sklearn.base import BaseEstimator, TransformerMixin

class FeatureSelection(BaseEstimator, TransformerMixin):
    
    def __init__(self,selected_features):
        self.selected_features=selected_features
    
    def fit(self,X,y=None):
        return self
    
    def transform(self, X, y=None):
        return X[self.selected_features]

**(ii)** Armar los pasos para el `Pipeline` e instancia el `Pipeline`

In [8]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
import numpy as np

In [9]:
selector=FeatureSelection(selected_features=X.drop(columns='residual sugar').columns.values)

In [10]:
pasos = [('feature_engineering',selector),
         ('preprocesamiento', MinMaxScaler()), 
         ('clasificador', LogisticRegression(solver='lbfgs'))]

pipe=Pipeline(pasos)

**(iii)** Generar la grilla y aplicar `GridSearchCV` con la `LogisticRegression`

In [11]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

folds=StratifiedKFold(n_splits=5,shuffle=True, random_state=42)

In [12]:
param_grid = [{'feature_engineering':[selector, None],
               'preprocesamiento':[StandardScaler(), MinMaxScaler()],
               'clasificador__C':np.linspace(0.1,1000,100)}] #list(np.linspace(0,1000,100)

grid=GridSearchCV(pipe, param_grid, cv=folds)

grid.fit(X_train,y_train)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             estimator=Pipeline(steps=[('feature_engineering',
                                        FeatureSelection(selected_features=array(['fixed acidity', 'volatile acidity', 'citric acid', 'chlorides',
       'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH',
       'sulphates', 'alcohol'], dtype=object))),
                                       ('preprocesamiento', MinMaxScaler(...
       9.091e+02, 9.192e+02, 9.293e+02, 9.394e+02, 9.495e+02, 9.596e+02,
       9.697e+02, 9.798e+02, 9.899e+02, 1.000e+03]),
                          'feature_engineering': [FeatureSelection(selected_features=array(['fixed acidity', 'volatile acidity', 'citric acid', 'chlorides',
       'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH',
       'sulphates', 'alcohol'], dtype=object)),
                                                  None],
                          'preprocesamiento': [StandardSca

**(iv)** Identificar cuál fue el mejor modelo, cuál fue su score de validación y evaluar su performance con los datos de test

In [13]:
grid.best_estimator_

Pipeline(steps=[('feature_engineering', None),
                ('preprocesamiento', MinMaxScaler()),
                ('clasificador', LogisticRegression(C=70.8))])

In [14]:
grid.best_params_

{'clasificador__C': 70.8,
 'feature_engineering': None,
 'preprocesamiento': MinMaxScaler()}

In [15]:
grid.best_score_

0.7354620435618193

In [16]:
grid.best_estimator_.score(X_test,y_test)

0.75625

<a href='#tabla_contenidos'>Volver a TOC</a>